In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import glob
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
import random

def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seeds(42)

In [3]:
image_original_path = r"C:/Users/ADMIN/Documents/ANkhe/v3_KaNak2_new/v3_KaNak2_new/KaNak2_new"
image_mask_path = r"C:/Users/ADMIN/Documents/ANkhe/v3_KaNak2_new/v3_KaNak2_new/masks"
label_path = r"C:/Users/ADMIN/Documents/ANkhe/v3_KaNak2_new/v3_KaNak2_new/annotations.xml"
csv_path = r"C:/Users/ADMIN/Documents/ANkhe/KaNak.csv"

print(f"Image original path exists: {os.path.exists(image_original_path)}")
print(f"Image mask path exists: {os.path.exists(image_mask_path)}")
print(f"Label path exists: {os.path.exists(label_path)}")
print(f"CSV path exists: {os.path.exists(csv_path)}")

Image original path exists: True
Image mask path exists: True
Label path exists: True
CSV path exists: True


In [ ]:
import xml.etree.ElementTree as ET

def parse_cvat_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    recs = []
    for img_el in root.findall(".//image"):
        name = img_el.attrib["name"]
        w = int(float(img_el.attrib["width"]))
        h = int(float(img_el.attrib["height"]))
        objs = []
        for poly in img_el.findall("polyline"):
            label = poly.attrib.get("label", "")
            pts = []
            for p in poly.attrib.get("points", "").split(";"):
                p = p.strip()
                if not p:
                    continue
                xs, ys = p.split(",")
                pts.append((float(xs), float(ys)))
            if len(pts) >= 2:
                objs.append({"label": label, "points": pts})
        recs.append({"name": name, "width": w, "height": h, "objects": objs})
    return recs

def polyline_to_mask(width, height, points, close=True, thickness=None):
    mask = np.zeros((height, width), dtype=np.uint8)
    pts = np.array(points, dtype=np.float32)
    pts[:, 0] = np.clip(pts[:, 0], 0, width - 1)
    pts[:, 1] = np.clip(pts[:, 1], 0, height - 1)
    pts = np.round(pts).astype(np.int32)

    if close:
        if not (pts[0] == pts[-1]).all():
            pts = np.vstack([pts, pts[0]])
        cv2.fillPoly(mask, [pts], 255)
    else:
        if thickness is None: thickness = 2
        cv2.polylines(mask, [pts], isClosed=False, color=255, thickness=thickness)
    return mask

def build_binary_mask(rec, target_labels=None, close=True, thickness=None):
    w, h = rec["width"], rec["height"]
    mask = np.zeros((h, w), dtype=np.uint8)
    for obj in rec["objects"]:
        if (target_labels is None) or (obj["label"] in target_labels):
            m = polyline_to_mask(w, h, obj["points"], close=close, thickness=thickness)
            mask = np.maximum(mask, m)
    return (mask > 0).astype(np.uint8)

def load_masks_from_cvat_xml(xml_path, images_root=None, target_labels={"lake"}, target_size=(320,320)):
    """
    Trả về:
      masks: np.ndarray (N, H, W) đã resize về target_size và chuẩn hoá [0,1]
      filenames: list[str] tên ảnh tương ứng trong XML
    """
    recs = parse_cvat_xml(xml_path)

    masks = []
    names = []
    for rec in recs:
        mask_bin = build_binary_mask(rec, target_labels=target_labels, close=True)
        mask_resized = cv2.resize(mask_bin, target_size, interpolation=cv2.INTER_NEAREST)
        masks.append(mask_resized.astype(np.float32))  # 0/1
        names.append(os.path.basename(rec["name"]))
    return np.stack(masks, axis=0), names


In [ ]:
mask_images, image_filenames = load_masks_from_cvat_xml(
    xml_path=label_path,
    # images_root=image_original_path,    
    target_labels={"lake"},
    target_size=(320, 320)
)

print(f"Loaded {len(mask_images)} masks from XML")
print(f"Mask array shape: {mask_images.shape}  # (N, 320, 320)")



Loaded 32 masks from XML
Mask array shape: (32, 320, 320)  # (N, 320, 320)


In [6]:
def interpolate_images_to_monthly(images, target_months=48):
    if len(images) >= target_months:
        return images[:target_months]

    interpolated = []
    images_per_month = len(images) / 12.0

    for month in range(target_months):
        year = month // 12
        month_in_year = month % 12

        base_idx = int(month_in_year * images_per_month) % len(images)
        base_image = images[base_idx].copy()

        seasonal_noise = 0.05 * np.sin(2 * np.pi * month_in_year / 12)
        noise = np.random.normal(0, 0.02, base_image.shape)

        interpolated_image = np.clip(base_image + seasonal_noise + noise, 0, 1)
        interpolated.append(interpolated_image)

    return np.array(interpolated)

interpolated_images = interpolate_images_to_monthly(mask_images, 48)
interpolated_images = interpolated_images.reshape(-1, 1, 320, 320)
print(f"Interpolated images shape: {interpolated_images.shape}")

Interpolated images shape: (48, 1, 320, 320)


In [7]:
df = pd.read_csv(csv_path)
df['Time'] = pd.to_datetime(df['Time'])
df = df.sort_values('Time').reset_index(drop=True)

df.columns = ['Time', 'WaterLevel_m', 'TotalDischarge_m3s', 'Inflow_m3s']

df['Month'] = df['Time'].dt.month
df['IsFloodSeason'] = ((df['Month'] >= 5) & (df['Month'] <= 10)).astype(int)

for lag in [1, 2, 3]:
    df[f'WaterLevel_lag{lag}'] = df['WaterLevel_m'].shift(lag)
    df[f'Inflow_lag{lag}'] = df['Inflow_m3s'].shift(lag)

df = df.dropna().reset_index(drop=True)
print(f"Time series data shape: {df.shape}")
print(f"Date range: {df['Time'].min()} to {df['Time'].max()}")

Time series data shape: (35000, 12)
Date range: 2019-01-01 03:00:00 to 2022-12-31 23:00:00


In [8]:
%cd Pytorch-UNet

c:\Users\ADMIN\Documents\ANkhe\Pytorch-UNet


In [9]:
import torch
import torch.nn as nn
from unet import UNet, DynamicSnake


class GeM(nn.Module):
    """Generalized Mean Pooling; p=1 -> GAP, p->inf -> GMP"""
    def __init__(self, p: float = 3.0, eps: float = 1e-6):
        super().__init__()
        self.p = nn.Parameter(torch.tensor(p))
        self.eps = eps
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.clamp(min=self.eps).pow(self.p)
        return x.mean(dim=(-2, -1)).pow(1.0 / self.p)


class UNetEDMSFeatureExtractor(nn.Module):
    """
    Multi-scale features from BOTH encoder and decoder:
      Encoder: x3, x4, x5
      Decoder: d1, d2, d3, d4 (sau mỗi Up)

    freeze_backbone=True + unfreeze_dynamic_snake=True:
      - Đóng băng backbone ngoại trừ các tham số của DynamicSnake (alpha).
    """
    def __init__(
        self,
        input_channels: int = 1,
        output_features: int = 1024,
        n_classes: int = 2,
        bilinear: bool = False,
        freeze_backbone: bool = True,
        unfreeze_dynamic_snake: bool = True,
        use_encoder_stages = ("x3", "x4", "x5"),
        use_decoder_stages = ("d1", "d2", "d3", "d4"),
        act_type_enc: str = "relu",
        act_type_dec: str = "snake",
        **unet_kwargs,
    ):
        super().__init__()

        self.unet = UNet(
            n_channels=input_channels,
            n_classes=n_classes,
            bilinear=bilinear,
            act_type_enc=act_type_enc,
            act_type_dec=act_type_dec,
            **unet_kwargs,
        )

        self.pool = GeM(p=3.0)

        with torch.no_grad():
            dummy = torch.zeros(1, input_channels, 320, 320)
            x1 = self.unet.inc(dummy)
            x2 = self.unet.down1(x1)
            x3 = self.unet.down2(x2)
            x4 = self.unet.down3(x3)
            x5 = self.unet.down4(x4)
            d1 = self.unet.up1(x5, x4)
            d2 = self.unet.up2(d1, x3)
            d3 = self.unet.up3(d2, x2)
            d4 = self.unet.up4(d3, x1)

            ch = {
                "x3": x3.shape[1], "x4": x4.shape[1], "x5": x5.shape[1],
                "d1": d1.shape[1], "d2": d2.shape[1], "d3": d3.shape[1], "d4": d4.shape[1],
            }

        self.use_encoder_stages = tuple(use_encoder_stages)
        self.use_decoder_stages = tuple(use_decoder_stages)

        concat_dim = sum(ch[k] for k in self.use_encoder_stages) + \
                     sum(ch[k] for k in self.use_decoder_stages)

        self.norm = nn.LayerNorm(concat_dim)
        self.head = nn.Sequential(
            nn.Linear(concat_dim, 1024), nn.ReLU(inplace=True),
            nn.Linear(1024, output_features), nn.ReLU(inplace=True),
        )

        if freeze_backbone:
            for p in self.unet.parameters():
                p.requires_grad = False

            if unfreeze_dynamic_snake:
                for m in self.unet.modules():
                    if isinstance(m, DynamicSnake):
                        for p in m.parameters():
                            p.requires_grad = True

    @torch.no_grad()
    def forward(self, x: torch.Tensor, return_dict: bool = False):
        x1 = self.unet.inc(x)
        x2 = self.unet.down1(x1)
        x3 = self.unet.down2(x2)
        x4 = self.unet.down3(x3)
        x5 = self.unet.down4(x4)
        d1 = self.unet.up1(x5, x4)
        d2 = self.unet.up2(d1, x3)
        d3 = self.unet.up3(d2, x2)
        d4 = self.unet.up4(d3, x1)

        feat = {"x3": x3, "x4": x4, "x5": x5, "d1": d1, "d2": d2, "d3": d3, "d4": d4}

        zs = [self.pool(feat[k]) for k in self.use_encoder_stages]
        zs += [self.pool(feat[k]) for k in self.use_decoder_stages]

        z = torch.cat(zs, dim=1)
        z = self.norm(z)
        z = self.head(z)

        if return_dict:
            return z, feat
        return z


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

unet_model = UNetEDMSFeatureExtractor(
    input_channels=1,
    output_features=1024,
    n_classes=2,
    bilinear=False,
    freeze_backbone=True,          # hoặc False nếu muốn fine-tune toàn bộ
    unfreeze_dynamic_snake=True,   # để DynamicSnake còn học alpha
    act_type_enc="relu",
    act_type_dec="relu",
    c3str_layers=2, c3str_heads=8, c3str_mlp_ratio=4, c3str_dropout=0.1
).to(device)


In [11]:
def extract_image_features(model, images, batch_size=4):
    model.eval()
    feats = []
    with torch.no_grad():
        for i in range(0, len(images), batch_size):
            batch = torch.as_tensor(images[i:i+batch_size], dtype=torch.float32, device=device)
            z = model(batch)
            feats.append(z.cpu().numpy())
    return np.vstack(feats)

image_features = extract_image_features(unet_model, interpolated_images, batch_size=4)
print(f"Image features shape: {image_features.shape}")


Image features shape: (48, 1024)


In [12]:
class TimeSeriesFeatureExtractor(nn.Module):
    def __init__(self, input_dim, output_features=1024):
        super(TimeSeriesFeatureExtractor, self).__init__()

        self.fc_layers = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, output_features),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.fc_layers(x)

feature_columns = ['Inflow_m3s', 'TotalDischarge_m3s', 'IsFloodSeason',
                  'WaterLevel_lag1', 'Inflow_lag1', 'WaterLevel_lag2',
                  'Inflow_lag2', 'WaterLevel_lag3', 'Inflow_lag3']

ts_feature_data = df[feature_columns].values
ts_scaler = StandardScaler()
ts_feature_data_scaled = ts_scaler.fit_transform(ts_feature_data)

ts_model = TimeSeriesFeatureExtractor(len(feature_columns)).to(device)

def extract_ts_features(model, data, batch_size=32):
    model.eval()
    features = []

    with torch.no_grad():
        for i in range(0, len(data), batch_size):
            batch = data[i:i+batch_size]
            batch_tensor = torch.FloatTensor(batch).to(device)
            batch_features = model(batch_tensor)
            features.append(batch_features.cpu().numpy())

    return np.vstack(features)

ts_features = extract_ts_features(ts_model, ts_feature_data_scaled)
print(f"Time series features shape: {ts_features.shape}")

Time series features shape: (35000, 1024)


In [13]:
def expand_image_features_to_daily(image_features, n_days):
    n_months = len(image_features)
    days_per_month = n_days / n_months

    expanded_features = []
    for i, monthly_feature in enumerate(image_features):
        start_day = int(i * days_per_month)
        end_day = int((i + 1) * days_per_month)

        for _ in range(end_day - start_day):
            expanded_features.append(monthly_feature)

    return np.array(expanded_features[:n_days])

expanded_image_features = expand_image_features_to_daily(image_features, len(ts_features))
print(f"Expanded image features shape: {expanded_image_features.shape}")

Expanded image features shape: (35000, 1024)


In [14]:
feature_scaler = StandardScaler()
image_features_norm = feature_scaler.fit_transform(expanded_image_features)
ts_features_norm = feature_scaler.fit_transform(ts_features)

combined_features = np.concatenate([image_features_norm, ts_features_norm], axis=1)
print(f"Combined features shape: {combined_features.shape}")

Combined features shape: (35000, 2048)


In [15]:
def create_sequences_for_gru(features, labels, time_steps=4):
    X, y = [], []

    for i in range(time_steps, len(features)):
        X.append(features[i-time_steps:i])
        y.append(labels[i])

    return np.array(X), np.array(y)

labels = df['WaterLevel_m'].values
min_len = min(len(combined_features), len(labels))
combined_features = combined_features[:min_len]
labels = labels[:min_len]

X_seq, y_seq = create_sequences_for_gru(combined_features, labels, time_steps=4)
print(f"Sequence shapes - X: {X_seq.shape}, y: {y_seq.shape}")

Sequence shapes - X: (34996, 4, 2048), y: (34996,)


In [16]:
train_size = int(0.7 * len(X_seq))
val_size = int(0.2 * len(X_seq))

X_train = X_seq[:train_size]
y_train = y_seq[:train_size]
X_val = X_seq[train_size:train_size+val_size]
y_val = y_seq[train_size:train_size+val_size]
X_test = X_seq[train_size+val_size:]
y_test = y_seq[train_size+val_size:]

print(f"Train: {len(X_train)}, Validation: {len(X_val)}, Test: {len(X_test)}")

baseline_ts_sequences, _ = create_sequences_for_gru(ts_features, labels, time_steps=4)
baseline_X_train = baseline_ts_sequences[:train_size]
baseline_X_val = baseline_ts_sequences[train_size:train_size+val_size]
baseline_X_test = baseline_ts_sequences[train_size+val_size:]

Train: 24497, Validation: 6999, Test: 3500


In [17]:
class WaterLevelDataset(Dataset):
    def __init__(self, sequences, targets, scaler_y=None, fit_scaler=False):
        self.sequences = torch.FloatTensor(sequences)

        if fit_scaler:
            if scaler_y is None:
                self.scaler_y = MinMaxScaler()
            else:
                self.scaler_y = scaler_y
            targets_scaled = self.scaler_y.fit_transform(targets.reshape(-1, 1)).flatten()
        elif scaler_y is not None:
            self.scaler_y = scaler_y
            targets_scaled = scaler_y.transform(targets.reshape(-1, 1)).flatten()
        else:
            self.scaler_y = None
            targets_scaled = targets

        self.targets = torch.FloatTensor(targets_scaled)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]

scaler_y = MinMaxScaler()
train_dataset = WaterLevelDataset(X_train, y_train, scaler_y, fit_scaler=True)
val_dataset = WaterLevelDataset(X_val, y_val, scaler_y)
test_dataset = WaterLevelDataset(X_test, y_test, scaler_y)

baseline_train_dataset = WaterLevelDataset(baseline_X_train, y_train, scaler_y)
baseline_val_dataset = WaterLevelDataset(baseline_X_val, y_val, scaler_y)
baseline_test_dataset = WaterLevelDataset(baseline_X_test, y_test, scaler_y)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

baseline_train_loader = DataLoader(baseline_train_dataset, batch_size=32, shuffle=True)
baseline_val_loader = DataLoader(baseline_val_dataset, batch_size=32, shuffle=False)
baseline_test_loader = DataLoader(baseline_test_dataset, batch_size=32, shuffle=False)

In [18]:
class ProposedGRUModel(nn.Module):
    def __init__(self, feature_dim, hidden_dim=128, num_layers=2):
        super(ProposedGRUModel, self).__init__()

        self.gru = nn.GRU(feature_dim, hidden_dim, num_layers,
                         batch_first=True, dropout=0.2)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 50),
            nn.ReLU(),
            nn.Linear(50, 1)
        )

    def forward(self, x):
        gru_out, _ = self.gru(x)
        last_output = gru_out[:, -1, :]
        output = self.dropout(last_output)
        output = self.fc(output)
        return output

class BaselineGRUModel(nn.Module):
    def __init__(self, feature_dim, hidden_dim=64, num_layers=2):
        super(BaselineGRUModel, self).__init__()

        self.gru = nn.GRU(feature_dim, hidden_dim, num_layers,
                         batch_first=True, dropout=0.2)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 25),
            nn.ReLU(),
            nn.Linear(25, 1)
        )

    def forward(self, x):
        gru_out, _ = self.gru(x)
        last_output = gru_out[:, -1, :]
        output = self.dropout(last_output)
        output = self.fc(output)
        return output

proposed_model = ProposedGRUModel(X_seq.shape[2]).to(device)
baseline_model = BaselineGRUModel(baseline_ts_sequences.shape[2]).to(device)

print("Models built")
print(f"Proposed model input shape: {X_seq.shape}")
print(f"Baseline model input shape: {baseline_ts_sequences.shape}")


Models built
Proposed model input shape: (34996, 4, 2048)
Baseline model input shape: (34996, 4, 1024)


In [19]:
def train_model(model, train_loader, val_loader, num_epochs=50, lr=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for sequences, targets in train_loader:
            sequences, targets = sequences.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(sequences).squeeze()
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for sequences, targets in val_loader:
                sequences, targets = sequences.to(device), targets.to(device)
                outputs = model(sequences).squeeze()
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    return train_losses, val_losses

print("Training proposed model...")
proposed_train_losses, proposed_val_losses = train_model(proposed_model, train_loader, val_loader)

# print("Training baseline model...")model(baseline_model, baseline_train_loader, baseline_val_loader)


Training proposed model...
Epoch [10/50], Train Loss: 0.0000, Val Loss: 0.0001
Epoch [20/50], Train Loss: 0.0000, Val Loss: 0.0001
Epoch [30/50], Train Loss: 0.0000, Val Loss: 0.0001
Epoch [40/50], Train Loss: 0.0000, Val Loss: 0.0001
Epoch [50/50], Train Loss: 0.0000, Val Loss: 0.0001


In [20]:
def make_predictions(model, test_loader, scaler_y):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for sequences, targets in test_loader:
            sequences = sequences.to(device)
            outputs = model(sequences).squeeze()
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(targets.numpy())

    predictions = np.array(predictions)
    actuals = np.array(actuals)

    predictions_unscaled = scaler_y.inverse_transform(predictions.reshape(-1, 1)).flatten()
    actuals_unscaled = scaler_y.inverse_transform(actuals.reshape(-1, 1)).flatten()

    return predictions_unscaled, actuals_unscaled

y_pred_proposed, y_test_actual = make_predictions(proposed_model, test_loader, scaler_y)
y_pred_baseline, _ = make_predictions(baseline_model, baseline_test_loader, scaler_y)

print("Predictions completed")
print(f"Predicted shapes - Proposed: {y_pred_proposed.shape}, Baseline: {y_pred_baseline.shape}")
print(f"Actual test shape: {y_test_actual.shape}")


Predictions completed
Predicted shapes - Proposed: (3500,), Baseline: (3500,)
Actual test shape: (3500,)


In [21]:
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return {'MAE': mae, 'MSE': mse, 'RMSE': rmse}

proposed_metrics = calculate_metrics(y_test_actual, y_pred_proposed)
baseline_metrics = calculate_metrics(y_test_actual, y_pred_baseline)

mae_improvement = (baseline_metrics['MAE'] - proposed_metrics['MAE']) / baseline_metrics['MAE'] * 100
mse_improvement = (baseline_metrics['MSE'] - proposed_metrics['MSE']) / baseline_metrics['MSE'] * 100
rmse_improvement = (baseline_metrics['RMSE'] - proposed_metrics['RMSE']) / baseline_metrics['RMSE'] * 100

print("BASELINE GRU MODEL RESULTS:")
print(f"MAE:  {baseline_metrics['MAE']:.4f}")
print(f"MSE:  {baseline_metrics['MSE']:.4f}")
print(f"RMSE: {baseline_metrics['RMSE']:.4f}")

print(f"\nPROPOSED MODEL (unet + GRU) RESULTS:")
print(f"MAE:  {proposed_metrics['MAE']:.4f}")
print(f"MSE:  {proposed_metrics['MSE']:.4f}")
print(f"RMSE: {proposed_metrics['RMSE']:.4f}")

print(f"\nIMPROVEMENT OVER BASELINE:")
print(f"MAE improvement:  {mae_improvement:.1f}%")
print(f"MSE improvement:  {mse_improvement:.1f}%")
print(f"RMSE improvement: {rmse_improvement:.1f}%")


BASELINE GRU MODEL RESULTS:
MAE:  111.1228
MSE:  12374.1572
RMSE: 111.2392

PROPOSED MODEL (unet + GRU) RESULTS:
MAE:  2.7099
MSE:  9.3517
RMSE: 3.0581

IMPROVEMENT OVER BASELINE:
MAE improvement:  97.6%
MSE improvement:  99.9%
RMSE improvement: 97.3%
